# Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
import requests
from bs4 import BeautifulSoup

**Scraping Wikipedia webpage**

In [12]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wiki.content, 'html.parser')
tables = soup.find_all('table',class_="wikitable")
column_names = [item.get_text() for item in tables[0].find_all('th')]
contents = [item.get_text() for item in tables[0].find_all('td')]
values=[]

for table in tables:
    for item in table.select('td'):
        temp = item.get_text()
        values.append(temp)
print(len(values)/3)

toronto_data = np.reshape(values,(288,3))

toronto_df = pd.DataFrame(data = toronto_data)

toronto_df.columns = ["Postal Code", "Borough", "Neighbourhood"]

toronto_df['Neighbourhood'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')

# filter column Neighbourhood

toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']

toronto_df.head()

288.0


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
# Group, concatenate & set values in 'Neighbourhood column' to values in 'Borough' if not assigned.

toronto_df_1 = toronto_df.groupby(['Postal Code', 'Borough'])['Neighbourhood'] \
                .apply(', '.join) \
                .reset_index()

toronto_df_1['Neighbourhood'].loc[toronto_df_1['Neighbourhood'] == 'Not assigned'] = toronto_df_1['Borough']

toronto_df_1.shape

(103, 3)

__Searching longtitude and latitude for the boroughs__

In [18]:
coordinate_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [19]:
toronto_df_final = pd.merge(toronto_df_1, coordinate_df, how = 'inner', on = 'Postal Code')